<a href="https://colab.research.google.com/github/dcthyun0308/ESAA/blob/main/ESAA_YB_2%EC%A1%B0_mini3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('movies_train.csv')
test = pd.read_csv('movies_test.csv')
sub = pd.read_csv('submission.csv')

In [3]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [4]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1


In [5]:
sub.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


#**전처리**

##**결측치 처리**

In [6]:
# train 결측치 확인
print(train.isnull().sum())

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64


In [7]:
# test 결측치 확인
print(test.isnull().sum())

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
dtype: int64


In [8]:
# dir_prev_bfunm 결측치를 0으로 변경
train['dir_prev_bfnum'] = train['dir_prev_bfnum'].fillna(0)
test['dir_prev_bfnum'] = test['dir_prev_bfnum'].fillna(0)

In [9]:
# 결측치 확인
print(train.isnull().sum(), '\n')
print(test.isnull().sum())

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
dtype: int64 

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
dtype: int64


##**날짜형 변수 처리**

In [10]:
# 개봉일을 연도, 월로 분리

# 문자열을 datetime으로 변환
train['release_time'] = pd.to_datetime(train['release_time'])
test['release_time'] = pd.to_datetime(test['release_time'])

# release_year, release_month
train['release_year'] = train['release_time'].dt.year
train['release_month'] = train['release_time'].dt.month
test['release_year'] = test['release_time'].dt.year
test['release_month'] = test['release_time'].dt.month

# release time 제거
train = train.drop(['release_time'], axis=1)
test = test.drop(['release_time'], axis=1)

##**이상치 로그변환**

In [11]:
#이상치 확인
Q1 = train['box_off_num'].quantile(0.25)
Q3 = train['box_off_num'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = train[(train['box_off_num'] < lower) | (train['box_off_num'] > upper)]
out_num = outliers['box_off_num']

print(f'관객수 이상치 개수: {len(out_num)}')

관객수 이상치 개수: 97


In [12]:
# 관객수 이상치 로그변환
train['box_off_num_log'] = np.log1p(train['box_off_num'])

##**배급사명 통일**

In [13]:
# 배급사값 확인
print(sorted(train['distributor'].unique()))
print(sorted(test['distributor'].unique()))

['(주) 브릿지웍스', '(주) 액티버스엔터테인먼트', '(주) 영화사조제', '(주) 영화제작전원사', '(주) 케이알씨지', '(주)JK필름', '(주)NEW', '(주)고앤고 필름', '(주)노버스엔터테인먼트', '(주)대명문화공장', '(주)더픽쳐스', '(주)두타연', '(주)드림팩트엔터테인먼트', '(주)랠리버튼', '(주)레인보우 팩토리', '(주)로드하우스', '(주)리틀빅픽처스', '(주)마운틴픽쳐스', '(주)마인스 엔터테인먼트', '(주)마인스엔터테인먼트', '(주)미디어데이', '(주)쇼박스', '(주)스마일이엔티', '(주)스톰픽쳐스코리아', '(주)시네마달', '(주)씨엠닉스', '(주)아우라픽처스', '(주)아이필름코퍼레이션', '(주)에이원 엔터테인먼트', '(주)엣나인필름', '(주)예지림 엔터테인먼트', '(주)온비즈넷', '(주)유비네트워크', '(주)유비콘텐츠', '(주)이스트스카이필름', '(주)콘텐츠 윙', '(주)키노아이', '(주)팝 파트너스', '(주)팝엔터테인먼트', '(주)패뷸러스', '(주)패스파인더씨앤씨', '(주)프레인글로벌', '(주)피터팬픽쳐스', '(주)휘엔터테인먼트', 'BoXoo 엔터테인먼트', 'CGV 무비꼴라쥬', 'CGV 아트하우스', 'CGV아트하우스', 'CJ CGV', 'CJ E&M Pictures', 'CJ E&M 영화부문', 'CJ 엔터테인먼트', 'KBS미디어(주)', 'KT', 'KT&G 상상마당', 'M2픽처스', 'NEW', 'OAL(올)', 'SBS콘텐츠허브', 'SK텔레콤(주)', 'SK플래닛', 'THE 픽쳐스', 'tvN', 'ysfilm', '건시네마', '고구마공작소', '골든타이드픽처스', '김기덕 필름', '나우콘텐츠', '나이너스엔터테인먼트(주)', '노바엔터테인먼트', '더 피플', '더픽쳐스/(주)마운틴픽쳐스', '동국대학교 충무로영상제작센터', '두 엔터테인먼트', '드림로드', '드림팩트 엔터테인먼트', '디 씨드', '디마엔터테인먼트', '롤러코스터 

In [14]:
train['distributor'].value_counts()

,count
distributor,
CJ 엔터테인먼트,54
롯데엔터테인먼트,52
(주)NEW,30
(주)마운틴픽쳐스,29
(주)쇼박스,26
...,...
영화사 廊,1
크리에이티브컴즈(주),1
ysfilm,1


In [15]:
test['distributor'].value_counts()

,count
distributor,
CJ 엔터테인먼트,31
롯데엔터테인먼트,24
(주)마운틴픽쳐스,14
(주)쇼박스,12
골든타이드픽처스,12
...,...
(주)스톰픽쳐스코리아,1
(주)다세포클럽,1
디씨드,1


In [16]:
# 배급사명 통일
dist_map = {
    'CJ 엔터테인먼트': 'CJE&M Movie',
    'CJ E&M 영화부문': 'CJE&M Movie',
    'CJ E&M Pictures': 'CJE&M Movie',

    '(주)쇼박스': '쇼박스',
    '쇼박스(주)미디어플렉스': '쇼박스',

    '(주)NEW': 'NEW',

    '리틀빅픽처스': '리틀빅픽처스',
    '리틀빅픽쳐스': '리틀빅픽처스',

    '스폰지': '스폰지이엔티',
    '스폰지이엔티': '스폰지이엔티',

    '시네마 달': '시네마달',

    '드림팩트 엔터테인먼트': '드림팩트엔터테인먼트',
    '(주)드림팩트엔터테인먼트': '드림팩트엔터테인먼트',

    '(주)마운틴픽쳐스': '마운틴픽처스',
    '마운틴 픽처스': '마운틴픽처스',
    '마운틴픽쳐스': '마운틴픽처스',

    '(주)에이원 엔터테인먼트': '에이원엔터테인먼트',
    '에이원 엔터테인먼트': '에이원엔터테인먼트',

    '롯데쇼핑(주)롯데엔터테인먼트': '롯데엔터테인먼트',

    'CGV 아트하우스': 'CGV아트하우스',
    'CGV아트하우스': 'CGV아트하우스',
    'CGV 무비꼴라쥬': 'CGV아트하우스',
    '무비꼴라쟈쥬': 'CGV아트하우스',

    '싸이더스': '싸이더스FNH',
    '싸이더스 FNH': '싸이더스FNH',
    '싸이더스FNH': '싸이더스FNH',

    '(주) 마인스 엔터테인먼트': '(주)마인스엔터테인먼트',
}

# 매핑 적용
train['distributor'] = train['distributor'].replace(dist_map)
test['distributor'] = test['distributor'].replace(dist_map)

# 확인
print(sorted(train['distributor'].unique()))
print(sorted(test['distributor'].unique()))

['(주) 브릿지웍스', '(주) 액티버스엔터테인먼트', '(주) 영화사조제', '(주) 영화제작전원사', '(주) 케이알씨지', '(주)JK필름', '(주)고앤고 필름', '(주)노버스엔터테인먼트', '(주)대명문화공장', '(주)더픽쳐스', '(주)두타연', '(주)랠리버튼', '(주)레인보우 팩토리', '(주)로드하우스', '(주)리틀빅픽처스', '(주)마인스 엔터테인먼트', '(주)마인스엔터테인먼트', '(주)미디어데이', '(주)스마일이엔티', '(주)스톰픽쳐스코리아', '(주)시네마달', '(주)씨엠닉스', '(주)아우라픽처스', '(주)아이필름코퍼레이션', '(주)엣나인필름', '(주)예지림 엔터테인먼트', '(주)온비즈넷', '(주)유비네트워크', '(주)유비콘텐츠', '(주)이스트스카이필름', '(주)콘텐츠 윙', '(주)키노아이', '(주)팝 파트너스', '(주)팝엔터테인먼트', '(주)패뷸러스', '(주)패스파인더씨앤씨', '(주)프레인글로벌', '(주)피터팬픽쳐스', '(주)휘엔터테인먼트', 'BoXoo 엔터테인먼트', 'CGV아트하우스', 'CJ CGV', 'CJE&M Movie', 'KBS미디어(주)', 'KT', 'KT&G 상상마당', 'M2픽처스', 'NEW', 'OAL(올)', 'SBS콘텐츠허브', 'SK텔레콤(주)', 'SK플래닛', 'THE 픽쳐스', 'tvN', 'ysfilm', '건시네마', '고구마공작소', '골든타이드픽처스', '김기덕 필름', '나우콘텐츠', '나이너스엔터테인먼트(주)', '노바엔터테인먼트', '더 피플', '더픽쳐스/(주)마운틴픽쳐스', '동국대학교 충무로영상제작센터', '두 엔터테인먼트', '드림로드', '드림팩트엔터테인먼트', '디 씨드', '디마엔터테인먼트', '롤러코스터 프로덕션', '롯데엔터테인먼트', '리틀빅픽처스', '마노엔터테인먼트', '마법사필름', '마운틴픽처스', '머스트 씨 무비', '메가박스(주)플러스엠', '무비꼴라쥬', '미라클 필름', '백두대간', '보람엔터테인먼트', 

In [17]:
train['distributor'].value_counts()

,count
distributor,
CJE&M Movie,59
롯데엔터테인먼트,52
NEW,38
마운틴픽처스,32
쇼박스,28
...,...
영화사 廊,1
크리에이티브컴즈(주),1
ysfilm,1


In [18]:
test['distributor'].value_counts()

,count
distributor,
CJE&M Movie,32
롯데엔터테인먼트,25
마운틴픽처스,15
쇼박스,14
골든타이드픽처스,12
...,...
(주)스톰픽쳐스코리아,1
(주)다세포클럽,1
디씨드,1


##**라벨인코딩**

In [19]:
cols = ['distributor', 'genre', 'screening_rat', 'director']

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in cols:
    le = LabelEncoder()

    # train + test 묶어서 fit
    combined = pd.concat([train[col], test[col]]).astype(str)

    le.fit(combined)

    # transform 각각 적용
    train[col + '_le'] = le.transform(train[col].astype(str))
    test[col + '_le'] = le.transform(test[col].astype(str))

    label_encoders[col] = le

In [21]:
train.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,release_year,release_month,box_off_num_log,distributor_le,genre_le,screening_rat_le,director_le
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,0.00,0,91,2,23398,2012,11,10.060449,87,10,3,517
1,내부자들,쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,2015,11,15.771725,110,2,3,327
2,은밀하게 위대하게,쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083,2013,6,15.755558,110,10,1,467
3,나는 공무원이다,NEW,코미디,101,전체 관람가,구자홍,23894.00,2,20,6,217866,2012,7,12.291640,57,11,2,26
4,불량남녀,쇼박스,코미디,108,15세 관람가,신근호,1.00,1,251,2,483387,2010,11,13.088575,110,11,1,282


##**원-핫 인코딩**

In [22]:
# X, y 분리
X = train.drop(['box_off_num', 'title', 'box_off_num_log'], axis=1)
y = train['box_off_num']

# '기타'를 NaN으로 대체
X = X.replace('기타', np.nan)
test = test.replace('기타', np.nan)

# 3) train과 test를 합쳐서 one-hot 인코딩
full = pd.concat([X, test], axis=0)

# 4) 한 번에 get_dummies
full_encoded = pd.get_dummies(full, drop_first=True)

# 5) 다시 train/test로 나누기
X_encoded = full_encoded.iloc[:len(X), :]
test_encoded = full_encoded.iloc[len(X):, :]

print("X_encoded:", X_encoded.shape)
print("test_encoded:", test_encoded.shape)

X_encoded: (600, 1067)
test_encoded: (243, 1067)


1. LinearRegression, Ridge, Lasso, ElasticNet, RandomForest, GradientBoosting,  XGBoost, LightGBM 성능 비교

In [32]:
# import
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# train/validation 분리
X_tr, X_val, y_tr, y_val = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print("X_tr:", X_tr.shape, "X_val:", X_val.shape)

# 모델 리스트 선언
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.001),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5),
    "RandomForest": RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror'
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=300,
        learning_rate=0.1,
        random_state=42
    )
}

# 각 모델 학습 & 예측 & RMSE 계산
results = []

for name, model in models.items():
    # LightGBM은 feature name 때문에 DataFrame 대신 numpy array로 넣어줌
    if name == "LightGBM":
        model.fit(X_tr.values, y_tr)
        pred = model.predict(X_val.values)
    else:
        model.fit(X_tr, y_tr)
        pred = model.predict(X_val)

    # sklearn 버전에 따라 squared 인자 문제 있을 수 있어서 **0.5로 RMSE 계산
    rmse = mean_squared_error(y_val, pred) ** 0.5
    results.append((name, rmse))
    print(f"{name} RMSE: {rmse:.4f}")

# 결과를 표로 정리
results_df = pd.DataFrame(results, columns=["Model", "RMSE"])
results_df = results_df.sort_values(by="RMSE").reset_index(drop=True)
results_df


X_tr: (480, 1067) X_val: (120, 1067)
LinearRegression RMSE: 6031635.1695
Ridge RMSE: 1200494.4592
Lasso RMSE: 5740301.5554
ElasticNet RMSE: 1354787.7980
RandomForest RMSE: 1058368.2707
GradientBoosting RMSE: 1093461.9210
XGBoost RMSE: 1277859.0223
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000335 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 597
[LightGBM] [Info] Number of data points in the train set: 480, number of used features: 27
[LightGBM] [Info] Start training from score 723519.229167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

,Model,RMSE
0,RandomForest,1.058368e+06
1,GradientBoosting,1.093462e+06
2,Ridge,1.200494e+06
3,XGBoost,1.277859e+06
4,ElasticNet,1.354788e+06
5,LightGBM,1.614546e+06
6,Lasso,5.740302e+06
7,LinearRegression,6.031635e+06


2-1. RandomForest 튜닝 루트

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

rf_params = {
    'n_estimators': [200, 400],
    'max_depth': [None, 10],
    'min_samples_leaf': [1, 2]
}

rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)

rf_grid = GridSearchCV(
    rf_model,
    rf_params,
    cv=5,
    scoring='neg_mean_squared_error'
)

rf_grid.fit(X_encoded, y)

print("Best RF Params:", rf_grid.best_params_)
print("Best RF RMSE:", (-rf_grid.best_score_) ** 0.5)

# test 예측
pred_rf = rf_grid.best_estimator_.predict(test_encoded)

sub_rf = sub.copy()
sub_rf['box_off_num'] = pred_rf
sub_rf.to_csv('submission_rf.csv', index=False)

Best RF Params: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 400}
Best RF RMSE: 1435331.4954259729


2-2. 로그 타깃 LightGBM

In [37]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

y_log = np.log1p(y)

lgb_params = {
    'n_estimators': [200, 400],
    'learning_rate': [0.05, 0.1],
    'max_depth': [-1, 5]
}

lgb_model = LGBMRegressor(random_state=42)

# LightGBM용으로 컬럼 이름 안전하게 변경
X_lgb = X_encoded.copy()
test_lgb = test_encoded.copy()

X_lgb.columns = [f"f_{i}" for i in range(X_lgb.shape[1])]
test_lgb.columns = X_lgb.columns  # train이랑 똑같이 맞추기

lgb_grid = GridSearchCV(
    lgb_model,
    lgb_params,
    cv=5,
    scoring='neg_mean_squared_error'
)

lgb_grid.fit(X_lgb, y_log)

print("Best LGB Params:", lgb_grid.best_params_)
print("Best LGB RMSE(log):", (-lgb_grid.best_score_) ** 0.5)

# test 예측 → 역변환
pred_lgb_log = lgb_grid.best_estimator_.predict(test_encoded)
pred_lgb = np.expm1(pred_lgb_log)

sub_lgb = sub.copy()
sub_lgb['box_off_num'] = pred_lgb
sub_lgb.to_csv('submission_lgb_log.csv', index=False)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

2-3. 로그 타깃 XGBoost

In [38]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

y_log = np.log1p(y)

xgb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

xgb_model = XGBRegressor(
    random_state=42,
    objective='reg:squarederror'
)

xgb_grid = GridSearchCV(
    xgb_model,
    xgb_params,
    cv=5,
    scoring='neg_mean_squared_error'
)

xgb_grid.fit(X_encoded, y_log)

print("Best XGB Params:", xgb_grid.best_params_)
print("Best XGB RMSE(log):", (-xgb_grid.best_score_) ** 0.5)

pred_xgb_log = xgb_grid.best_estimator_.predict(test_encoded)
pred_xgb = np.expm1(pred_xgb_log)

sub_xgb = sub.copy()
sub_xgb['box_off_num'] = pred_xgb
sub_xgb.to_csv('submission_xgb_log.csv', index=False)


Best XGB Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best XGB RMSE(log): 1.800059164670168


2-결과정리. baseline보다 RMSE가 낮아야 좋은데
- RandomForest : 1,058,000 -> 1.435.331 성능하락
- LightGBM + log target : 1614545.6527 -> 1.7849855765617137(약 95~105만)
- XGBoost + log target : 1277859.0223 -> 1.800059164670168(약 100만 ~ 110만)

3. 앙상블

In [49]:
import numpy as np
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

y_log = np.log1p(y)   # 관객수 → 로그 스케일

# LightGBM이 싫어하는 특수문자 없게 컬럼 이름 단순화
X_ens = X_encoded.copy()
test_ens = test_encoded.copy()

X_ens.columns = [f"f_{i}" for i in range(X_ens.shape[1])]
test_ens.columns = X_ens.columns  # train/test 컬럼 이름 통일

lgb_best = LGBMRegressor(
    learning_rate=0.05,
    max_depth=-1,
    n_estimators=200,
    random_state=42
)

xgb_best = XGBRegressor(
    learning_rate=0.1,
    max_depth=5,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror"
)

lgb_best.fit(X_ens, y_log)
xgb_best.fit(X_ens, y_log)

lgb_log_pred_test = lgb_best.predict(test_ens)
xgb_log_pred_test = xgb_best.predict(test_ens)

lgb_pred_test = np.expm1(lgb_log_pred_test)
xgb_pred_test = np.expm1(xgb_log_pred_test)

w_lgb = 1 / 1.7849855765617137
w_xgb = 1 / 1.800059164670168
w_sum = w_lgb + w_xgb

ensemble_pred = (w_lgb * lgb_pred_test + w_xgb * xgb_pred_test) / w_sum

# 음수값 방지 (관객수이므로)
ensemble_pred = np.maximum(0, ensemble_pred)

sub['box_off_num'] = ensemble_pred
sub.to_csv("sample_submission_ensemble.csv", index=False)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 711
[LightGBM] [Info] Number of data points in the train set: 600, number of used features: 28
[LightGBM] [Info] Start training from score 9.951971
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be